# Docker

All the concepts were taken from [Основы Docker. Большой практический выпуск](https://youtu.be/QF4ZF857m44).

### Overview:
- [Introduction](#Introduction)
- [Getting started](#Getting-started)
- [Set up Dockerfile](#Set-up-Dockerfile)
- [Containerization](#Containerization) 
- [Work with volumes](#Work-with-volumes) 
- [Docker compose](#Docker-compose) 
- [Dockerhub](#Dockerhub) 

### Introduction

Docker provide us with simple way to pack, deliver and launch the application.
It is based on another type of virtualization in comparison with standard virtual machines: containers keep only services that are needed by an apllication. In order to do so there was introduced layers concept: instead of using the environment created on some server, we add to images necessary layers e.g. ubuntu + python + postgresql.
Such an approach allows us to create several instances of application with environments and run them in different places with the same logic of construction. 

### Getting started

In order to create an image of our application one might use the command `docker build -t hello-world .`, where `-t` stands for the tag of an image and `.` points out what is the directory to push. But the Docker should know what are the exact steps while creating the image, consequently we need to create so-called Dockerfile. 

Sample Dockerfile has the following form:
```
FROM python:3.6

RUN mkdir -p /usr/src/app/

WORKDIR /usr/src/app/

COPY . /usr/src/app/

RUN pip install --no-cache-dir -r requirements.txt

CMD["python", "app.py"]
```
The very first line indicates what is the base container to use, for this example we took Ubuntu with Python 3.6. Each of the lines creates new layer over the previous ones composing necessary environment.


### Set up Dockerfile

**Work with ports** <br>
`EXPOSE 8080` - expose port in order to access it from outside (see [how to run container](#create_run))

**Environment variables** <br>
`ENV <NAME> <VALUE>` - set environment variable, alternatively it can be [specified while running](#create_run)

### Containerization

**Create an image out of Dockerfile located in current directory** <br>
<a id="build"></a>
`docker build -t <TAG> <WORKDIR>`

**List all locally available images** <br>
`docker images` <br>
Remarkably that the `IMAGE ID` is the first 12 digits of a hash sum computed using SHA-256.

`-q` - list only ids of images

**Remove images** <br>
`docker rmi <NAME/ID>`

**Create and run the container** <br>
<a id="create_run"></a>
`docker run hello-world` 

`--name <name>` - specify name <br>
`-d` - run in the background <br>
`--rm` - remove after container stop <br>
`-p <PORT OF YOUR MACHINE>:<PORT INSIDE CONTAINER>` - work with specified port inside the container (requires `EXPOSE` in Dockerfile) <br>
`-e <VARNAME>=<VALUE>` - set value for a environment variable <br>
`-v <ABSOLUTE PATH ON HOST>:<ABSOLUTE PATH IN CONTAINER>` - mount the directory, so all local changes e.g in the directory will be reflected in the container <br>
`-v <VOLUME NAME>:<ABSOLUTE PATH IN CONTAINER>` - usually used for databases, mount the directory inside the container to volume 

**List running containers** <br>
`docker ps` 

`ps` stands for Process Status <br>
`-a` - list all containers <br>
`-q` - list only IDs of containers

**Remove containers** <br>
`docker rm <NAME/ID>`

! Hack: automatically remove all the containers :) <br>
`docker rm $(docker ps -qa)`

**Stop the container** <br> 
`docker stop <NAME/ID>`

### Work with volumes

Volumes used to save the information our application has produced. Indeed, while application is runnig all information placed by some path inside the container also goes to volume. Stored in volume information is accessible even after container removal. Mouting performed when [the container is runned](#create_run).   

**List all volumes** <br>
`docker volume ls`

**Create new volume** <br>
`docker volume create <NAME>`

### Docker compose

Development of large enough system often requires several microservices in different containers and interconnecting these containers with each other. However, it might quite complicated to set environment variables and run containers for microservices in `docker run` command. There Docker compose appears. File `docker-compose.yml` used to specify all the microservices Docker needs to run. In the following section we will provide sample structure of `docker-compose.yml`.

```
version: "3"

volumes: mongodb_volume

services:
    youtube_statistics:
        build: StatisticsManager/
        restart: always
        environment:
            - TZ=Europe/Moscow
            - MONGO_DB_ADDR=mongodb
            - MONGO_DB_PORT=27017
            ...
    
    web_service:
        build: WebService/
        restart: always
        ports:
            - 8080:8080
        environment:
            - MONGO_DB_ADDR=mongodb
            - MONGO_DB_PORT=27017
            - LOG_MODE=dev
            ...
    
    mongodb:
        image: mongo: latest
        volumes:
            - mongodb_volume: /data/db
        restart: always
        
```

`version: "3"` - declare format of Dockerfile (optional) <br>
`build: <PATH>` - specify directory with source code for microservice

**Run containers using Docker-compose** <br>
`docker-compose up`

### Dockerhub

**Push Docker container in Dockerhub** <br>
`docker build -t <USERNAME IN DOCKER>/<CONTAINER NAME> .` - ref. [docker build](#build) <br>
`docker push <USERNAME IN DOCKER>/<CONTAINER NAME>` 

Use `docker login` if necessary.